In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
## pip install xgboost

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#import xgboost as xgb




# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import h2o
import pandas as pd
from h2o.automl import H2OAutoML
h2o.init()

In [ ]:
data  = h2o.import_file("../input/breast-cancer-wisconsin-data/data.csv")

In [ ]:
type(data)

In [ ]:
data.columns
#data.drop(['id','Unnamed: 32'],axis =1, inplace = True)

In [ ]:
data['diagnosis'].unique()
#data['diagnosis'] = data['diagnosis'].map({'M':1,'B':0})

In [ ]:
data = data.drop('id',axis =1)

In [ ]:
data.head()

In [ ]:
# Identify predictors and response
x = data.columns
y = "diagnosis"
x.remove(y)

# For binary classification, response should be a factor
data[y] = data[y].asfactor()


# Run AutoML for 20 base models (limited to 1 hour max runtime by default)
aml = H2OAutoML(max_models=2, seed=1)
aml.train(x=x, y=y, training_frame=data)


In [ ]:

# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

In [ ]:
aml.leader

In [ ]:
########## H2o based neural network ##########

In [ ]:
from h2o.estimators.deeplearning import H2ODeepLearningEstimator

dl = H2ODeepLearningEstimator(activation="RectifierWithDropout",
                              hidden=[8,16,64],
                              input_dropout_ratio=0.2,
                              l1=1e-5,
                              epochs=10,
                              stopping_rounds=2)
dl.train(x,y,training_frame=data)

# Eval performance:
perf = dl.model_performance()

# Generate predictions on a test set (if necessary):
pred = dl.predict(data)

In [ ]:
pred.shape

In [ ]:
data.shape

In [ ]:
pred['predict'].unique()

In [ ]:
pred

In [ ]:
perf